## 📝 Lab #3 과제 정답

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, VotingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import plotly.express as px

# --- [문제 1] 데이터 로드 및 기본 전처리 ---
path = '../../datasets/ml/bike-sharing/SeoulBikeData.csv'
df_bike = pd.read_csv(path)

df_bike['Date'] = pd.to_datetime(df_bike['Date'], format='%d/%m/%Y')
df_bike['Month'] = df_bike['Date'].dt.month
df_bike['Day'] = df_bike['Date'].dt.day
df_bike = df_bike.drop('Date', axis=1)
df_bike = pd.get_dummies(df_bike, columns=['Seasons', 'Holiday', 'Functioning Day'], drop_first=True)
X = df_bike.drop('Rented Bike Count', axis=1)
y = df_bike['Rented Bike Count']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- [문제 2] 개별 모델 정의 ---
et_reg = ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1)
gbrt_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
xgb_reg = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=7, random_state=42, n_jobs=-1)

# --- [문제 3] VotingRegressor 정의 ---
voting_reg = VotingRegressor(estimators=[('et', et_reg), ('gbrt', gbrt_reg), ('xgb', xgb_reg)], n_jobs=-1)

# --- [문제 4] 학습 및 성능 비교 ---
models = [et_reg, gbrt_reg, xgb_reg, voting_reg]
model_names = ['ExtraTrees', 'GradientBoosting', 'XGBoost', 'Voting (Equal Weights)']

print("--- 모델별 RMSE 성능 비교 ---")
for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"{name}: {rmse:.4f}")

# --- [문제 5] 가중치를 적용한 VotingRegressor ---
# 개별 성능을 보니 XGBoost가 가장 우수하므로, 더 높은 가중치를 부여
weighted_voting_reg = VotingRegressor(
    estimators=[('et', et_reg), ('gbrt', gbrt_reg), ('xgb', xgb_reg)],
    weights=[0.15, 0.25, 0.6], # XGBoost에 높은 가중치
    n_jobs=-1
)
weighted_voting_reg.fit(X_train, y_train)
y_pred_weighted = weighted_voting_reg.predict(X_test)
rmse_weighted = np.sqrt(mean_squared_error(y_test, y_pred_weighted))
print(f"\nVoting (Weighted): {rmse_weighted:.4f}")


# --- [문제 6] 최종 모델 예측 결과 시각화 ---
# 최종 예측값을 데이터프레임으로 만들기
df_results = pd.DataFrame({'실제 대여량': y_test, '예측 대여량': y_pred_weighted})

fig = px.scatter(df_results,
                 x='실제 대여량',
                 y='예측 대여량',
                 title='실제 값 vs. 예측 값 (Weighted Voting)',
                 opacity=0.5,
                 trendline='ols', # Ordinary Least Squares 회귀선 추가
                 trendline_color_override='red'
                 )
fig.update_layout(
    xaxis_title="실제 대여량",
    yaxis_title="예측 대여량",
    width=800, height=600
)
fig.show()

# --- [문제 7] 결과 분석 ---
# 1. Voting 앙상블은 개별 모델 중 가장 성능이 좋았던 XGBoost와 비슷하거나 약간 더 나은 성능을 보여주었습니다. 이는 여러 모델의 예측을 평균내면서 일부 모델의 오차를 다른 모델이 상쇄해주는 효과 덕분입니다.
# 2. 성능이 좋은 모델에 더 높은 가중치를 부여했을 때, 앙상블 모델의 성능이 소폭 개선되었습니다. 이는 앙상블의 최종 결정에 더 정확한 모델의 "의견"을 더 많이 반영했기 때문입니다.
# 3. 산점도 그래프에서 점들이 y=x 직선(붉은색 추세선) 주위에 밀집할수록 예측이 정확함을 의미합니다.
#    - 대체로 추세선이 y=x와 비슷하게 그려져 전반적인 예측 경향은 좋습니다.
#    - 하지만 대여량이 매우 높은 값(2500 이상)에서는 점들이 추세선 아래에 위치하는 경향이 있는데, 이는 모델이 극단적으로 높은 수요를 과소예측하는 경향이 있음을 시사합니다.